問5<br>
いい機会なので、Gradient Boosting Decision Tree(GBDT)の一種であるXGBoost(eXtremeGradient Boosting)でやってみましょう。<br>
① 実際にやる前に、次の用語を調べてください。
- アンサンブル学習
- ブートストラップ
- バギング
- ブースティング

- アンサンブル学習<br>
精度の低い弱学習器を多く使うことで制度を上げる手法
- ブートストラップ<br>
重複を許して母集団からランダムにデータを流出する手法
- バギング<br>
ブースとトラップを使い抽出した複数のデータ群それぞれで並列的なモデルを構築し学習させる方法
- ブースティング<br>
すべての学習データを使って学習させたモデルを構築し、うまく予測できなかった学習データに対し重みづけをして再学習して新たなモデルを作る、というのを繰り返し、最終的なモデルを作成する

問5<br>
② XGBoostのパラメータについて調べてください。<br>
　 (役割、デフォルト値など)

・eta<br>
-過学習防止のための学習率の調整<br>
-値の範囲:0～1(デフォルト値:0.3)<br>
-小さくすると、モデルの分類性が向上<br>
・min_child_weight<br>
-決定木の葉の重みの下限 -> 下限が大きいほど単調な決定木になるため、過学習抑制<br>
-値の範囲:0～<br>
・max_depth<br>
-決定木の深さの最大値 -> 大きいと過学習しやすくなる<br>
-値の範囲:0～(デフォルト値:6)<br>
・gamma<br>
-決定木の追加による損失減少の下限 <- 設定値だけ損失関数が減少したらノードを分割する<br>
-値の範囲:0～(デフォルト値:0)

問6<br>
(まず)パラメータチューニングなしでXGBoostで予測をし精度検証を
してみてください。<br>
※ XGBoostの使い方は、SVCやランダムフォレストと同じです<br>
　(まずインポートします)->from xgboost import XGBClassifier<br>
　(次に、インスタンスを作ります)->xgb = XGBClassifier()<br>
　あとは'xgb'で.fit()します

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
cd /content/drive/MyDrive/AIStudy/git/exc_hyperpara_titanic

/content/drive/MyDrive/AIStudy/git/exc_hyperpara_titanic


In [31]:
df = pd.read_csv('./data/train.csv')
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [32]:
# 変数のSexとEmbarkedを数値に置き換える
# Embarkedの欠損はSで置き換える
df['Sex'] = df['Sex'].apply(lambda x: 0 if x=='male' else 1)
df['Embarked'] = df['Embarked'].fillna('S')
df['Embarked'] = df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1


In [33]:
# 使用する要素のみを抜き出す
df = df.drop(['PassengerId','Age','SibSp','Parch','Ticket','Cabin','Name'], axis=1)
df.head(2)

,Survived,Pclass,Sex,Fare,Embarked
0,0,3,0,7.2500,0
1,1,1,1,71.2833,1


In [34]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

x = df.drop('Survived', axis=1)
y = df['Survived']
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.3,random_state=0)

xgb = XGBClassifier()
xgb.fit(x_train, y_train)

XGBClassifier()

In [35]:
pred = xgb.predict(x_test)
notune_acc = accuracy_score(pred, y_test)
print('accuracy : {:.5f}' .format(notune_acc))

accuracy : 0.80597


問7<br>
XGBoostをグリッドサーチでチューニングしてみてください。<br>
やり方は、ランダムフォレストの時と同じですが、'params'で設定
したパラメータの内容で違うものがあります。<br>
問5で調べたものを
使ってください。

In [36]:
# 現在のハイパーパラメータの値を確認する
xgb.get_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 1,
 'verbosity': 1}

In [37]:
# グリッドサーチをする
from sklearn.model_selection import GridSearchCV
params = {
    'eta': [0.01, 0.1, 1.0],
    'gamma': [0, 0.1], 
    'n_estimators': [10, 100], 
    'max_depth':[2, 4], 
    'min_child_weigh': [1, 2]
    }
grid = GridSearchCV(estimator=XGBClassifier(), param_grid=params,cv=5, n_jobs=-1)
grid.fit(x_train, y_train)


GridSearchCV(cv=5, estimator=XGBClassifier(), n_jobs=-1,
             param_grid={'eta': [0.01, 0.1, 1.0], 'gamma': [0, 0.1],
                         'max_depth': [2, 4], 'min_child_weigh': [1, 2],
                         'n_estimators': [10, 100]})

In [38]:
print('best score : {:.5f}' .format(grid.best_score_))
print()
print('best parameters : {}' .format(grid.best_params_))

best score : 0.81057

best parameters : {'eta': 0.01, 'gamma': 0, 'max_depth': 4, 'min_child_weigh': 1, 'n_estimators': 100}


In [39]:
# テスト用データを用いて比較する
tune_xgb = XGBClassifier(eta = 0.01,gamma = 0, max_depth = 4,min_child_weight = 1, n_estimators=100)
tune_xgb.fit(x_train, y_train)
tune_pred = tune_xgb.predict(x_test)
tune_acc = accuracy_score(tune_pred, y_test)
print('ハイパーパラメータをチューニングしたスコア')
print('accuracy score : {:.5f}' .format(tune_acc))
print('ハイパーパラメータのチューニングをしていないスコア')
print('accuracy : {:.5f}' .format(notune_acc))

ハイパーパラメータをチューニングしたスコア
accuracy score : 0.81716
ハイパーパラメータのチューニングをしていないスコア
accuracy : 0.80597


問8<br>
グリッドサーチは、パラメータとして提示した値すべての組合せをしらみつぶしに探索する方法でした。<br>
このやり方はたいへん優れたやり方だとは思いますが、如何せん、組合せが増えると、探索に結構な時間を要する場合があります。<br>
これを解消する方法として、「ランダムサーチ」という方法があります。<br>
ランダムサーチは、しらみつぶしではなく、指定した回数だけランダムな組合せで探索行います。<br>コード的には、グリッドサーチとまったく同じで、GridSearchCVの代わりに、RandomizedSearchCVを使い、パラメータ'n_iter'でランダムサーチの回数を指定します.<br>

タイタニックでランダムサーチを試してみてください。

In [40]:
from sklearn.model_selection import RandomizedSearchCV
params = {
    'eta': [0.01, 0.1, 1.0],
    'gamma': [0, 0.1,0.001], 
    'n_estimators': [10, 100,1000], 
    'max_depth':[2, 4,6,8], 
    'min_child_weigh': [1, 2,3,4]
    }
rand = RandomizedSearchCV(estimator=XGBClassifier(), param_distributions=params,cv=5, n_jobs=-1,n_iter=30)
rand.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=XGBClassifier(), n_iter=30, n_jobs=-1,
                   param_distributions={'eta': [0.01, 0.1, 1.0],
                                        'gamma': [0, 0.1, 0.001],
                                        'max_depth': [2, 4, 6, 8],
                                        'min_child_weigh': [1, 2, 3, 4],
                                        'n_estimators': [10, 100, 1000]})

In [41]:
print('best score : {:.5f}' .format(rand.best_score_))
print()
print('best parameters : {}' .format(rand.best_params_))

best score : 0.81057

best parameters : {'n_estimators': 100, 'min_child_weigh': 1, 'max_depth': 4, 'gamma': 0, 'eta': 0.01}


In [ ]:
# 要素数を増やしたが結果は下がった
# 本来ベストスコアが出るものを実行していないと考えられる